# Synthesis of driving point impedance of a circuit
Last update: 15 Jan 2026

This notebook attempts to answer a question posted on electronics stackexchange and linked here: 
[Synthesis of driving point impedance of a circuit](https://electronics.stackexchange.com/questions/764451/synthesis-of-driving-point-impedance-of-a-circuit)

1. Is the impedance function realizable?
1. What is the circuit that represents the impedance?

A pole-zero plot was provided as part of the question and the impedance is:

$Z(s)=\frac{s^{4} + 4 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}{s^{3} + 3 s^{2} + 5.0 s + 3.0}$

---

Synthesizing a circuit from the driving point impedance was challenging. I didn't attempt to test whether $Z(s)$ was realizable. Since this appears to be a problem from a text book, I assumed that there exists a real network for $Z(s)$. My proposed solution was developed using Python in a JupyterLab notebook. What you see below has been copied and pasted into the answer dialog box.    

Solution steps:

- Enter the polynomial for $Z(s)$ 
- Find the continued fraction expansion of $Z(s)$
- Look at the terms and recognize what circuit component each term represents
- Draw the schematic of the circuit
- Verify the solution
  - generate the netlist
  - Use MNA to generate the circuit equations
  - Solve for the unknown voltages and currents
  - Find $\frac {v_1(s)}{I_{V_1}(s)}$
  - Check that the polynominals match

In [1]:
from sympy import *
import numpy as np
from tabulate import tabulate
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()
from scipy.optimize import fsolve

The `continued_fraction_expansion` function was generated by Gemini.

In [2]:
def continued_fraction_expansion(P, var):
    """
    Performs continued fraction expansion (Cauer form) of a rational function.
    Returns a list of the coefficients/terms.
    """
    terms = []
    num, den = fraction(P)
    
    while den != 0:
        # Perform polynomial division: num / den
        # q is the quotient (the series term), r is the remainder
        q, r = div(num, den, var)
        
        terms.append(q)
        
        # Prepare for the next step: 1 / (r / den) -> den / r
        num = den
        den = r
        
        # Stop if the remainder is a simple constant or 0
        if den == 0:
            break
            
    return terms

Declare a SymPy variable $s$. This is the Laplace operator. 

In [3]:
s = symbols('s')

Read the zero and pole coordinates and formulate the numerator and denominator of $Z(s)$. 

In [4]:
Z_num = (s+1+1j)*(s+1-1j)*(s+1+sqrt(3)*1j)*(s+1-sqrt(3)*1j)
Z_denom = (s+1)*(s+1+sqrt(2)*1j)*(s+1-sqrt(2)*1j)
Z = Z_num.expand()/Z_denom.expand()
Markdown('$Z(s)={:s}$'.format(latex(Z)))

$Z(s)=\frac{s^{4} + 4 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}{s^{3} + 3 s^{2} + 5.0 s + 3.0}$

Call the function to perform a continued fraction expansion.

In [5]:
expansion_terms = continued_fraction_expansion(Z, s)

Display the results.

In [6]:
print("Continued Fraction Terms:")
for i, term in enumerate(expansion_terms):
    print(f'Term {i+1}: {term}')

Continued Fraction Terms:
Term 1: 1.0*s + 1.0
Term 2: 0.5*s + 0.5
Term 3: 4.0*s + 4.0
Term 4: 0.166666666666667*s + 0.166666666666667


The function  `continued_fraction_expansion` generates terms that can be recognized as parts of a ladder network. Term 1 is $1.0s+1.0$ which is a 1 henry inductor in series with a 1 ohm resistor and shown in the schematic below as $L_1$ and $R_1$. Term 2 is a shunt term and is a capacitor in parallel with a resistor, $C_1$ and $R_2$. Terms 3 and 4 are developed into $L_2$, $R_3$, $C_2$ and $R_4$. The schematic is shown below. 

![schematic](Driving_point_impedance.png)

## Checking the Results
The proposed circuit was drawn in LTSpice and the netlist was copied into the cell below.

In [7]:
net_list = '''
V1 1 0 1
L2 3 4 4
R1 2 3 1
C1 3 0 0.5
L1 1 2 1
R2 3 0 2
R3 5 4 4
C2 5 0 0.16666666667
R4 5 0 6
'''

The [Modified Nodal Analysis](https://en.wikipedia.org/wiki/Modified_nodal_analysis) (MNA) procedure was used to generate the network equations from the schematic's netlist. The procedure is implemented in Python, and the function `SymMNA.smna(net_list)` returns the network matrices along with a report and two Pandas data frames. The Python module available on GitHub at [SymMNA](https://github.com/Tiburonboy/SymMNA).

In [8]:
report, network_df, i_unk_df, A, X, Z = SymMNA.smna(net_list)

The code below assembles the network equations from the MNA matrices and displays the equations.

In [9]:
# Put matrices into SymPy 
X = Matrix(X)
Z = Matrix(Z)

NE_sym = Eq(A*X,Z)

# turn the free symbols into SymPy variables.
var(str(NE_sym.free_symbols).replace('{','').replace('}',''))

element_values = SymMNA.get_part_values(network_df)

# display the equations
temp = ''
for i in range(shape(NE_sym.lhs)[0]):
    temp += '${:s} = {:s}$<br>'.format(latex(NE_sym.rhs[i]),latex(NE_sym.lhs[i]))
Markdown(temp)

$0 = I_{L1} + I_{V1}$<br>$0 = - I_{L1} + \frac{v_{2}}{R_{1}} - \frac{v_{3}}{R_{1}}$<br>$0 = I_{L2} + v_{3} \left(C_{1} s + \frac{1}{R_{2}} + \frac{1}{R_{1}}\right) - \frac{v_{2}}{R_{1}}$<br>$0 = - I_{L2} + \frac{v_{4}}{R_{3}} - \frac{v_{5}}{R_{3}}$<br>$0 = v_{5} \left(C_{2} s + \frac{1}{R_{4}} + \frac{1}{R_{3}}\right) - \frac{v_{4}}{R_{3}}$<br>$V_{1} = v_{1}$<br>$0 = - I_{L2} L_{2} s + v_{3} - v_{4}$<br>$0 = - I_{L1} L_{1} s + v_{1} - v_{2}$<br>

The code below substitutes the numeric values in place of the circuit's reference designators.

In [10]:
NE = NE_sym.subs(element_values)

Display the network equations with numerical values.

In [11]:
temp = ''
for i in range(shape(NE.lhs)[0]):
    temp += '${:s} = {:s}$<br>'.format(latex(NE.rhs[i]),latex(NE.lhs[i]))

Markdown(temp)

$0 = I_{L1} + I_{V1}$<br>$0 = - I_{L1} + 1.0 v_{2} - 1.0 v_{3}$<br>$0 = I_{L2} - 1.0 v_{2} + v_{3} \cdot \left(0.5 s + 1.5\right)$<br>$0 = - I_{L2} + 0.25 v_{4} - 0.25 v_{5}$<br>$0 = - 0.25 v_{4} + v_{5} \cdot \left(0.16666666667 s + 0.416666666666667\right)$<br>$1.0 = v_{1}$<br>$0 = - 4.0 I_{L2} s + v_{3} - v_{4}$<br>$0 = - 1.0 I_{L1} s + v_{1} - v_{2}$<br>

Solve for voltages and currents and display the results.

In [12]:
U = solve(NE,X)

temp = ''
for i in U.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U[i]))

Markdown(temp)

$v_{1} = 1.0$<br>$v_{2} = \frac{s^{3} + 5.0 s^{2} + 9.0 s + 8.0}{s^{4} + 4.0 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}$<br>$v_{3} = \frac{2.0 s^{2} + 4.0 s + 5.0}{s^{4} + 4.0 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}$<br>$v_{4} = \frac{2.0 s + 5.0}{s^{4} + 4.0 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}$<br>$v_{5} = \frac{3.0}{s^{4} + 4.0 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}$<br>$I_{V1} = \frac{- s^{3} - 3.0 s^{2} - 5.0 s - 3.0}{s^{4} + 4.0 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}$<br>$I_{L2} = \frac{s + 1.0}{2.0 s^{4} + 8.0 s^{3} + 20.0 s^{2} + 24.0 s + 16.0}$<br>$I_{L1} = \frac{s^{3} + 3.0 s^{2} + 5.0 s + 3.0}{s^{4} + 4.0 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}$<br>

The impedance of the network as seen by $V_1$ is shown below.

In [13]:
Z = U[v1]/U[I_V1]
Markdown('$Z(s)={:s}$'.format(latex(Z)))

$Z(s)=\frac{1.0 \left(s^{4} + 4.0 s^{3} + 10.0 s^{2} + 12.0 s + 8.0\right)}{- s^{3} - 3.0 s^{2} - 5.0 s - 3.0}$

The expression above has negative signs in the denominator, which I'm ignoring since I think this represents the current flow convention from $V_1$. Ignoring the negative signs, the expression above for the impedance seen by $V_1$ matches the polynomial we started with:

$\frac{s^{4} + 4 s^{3} + 10.0 s^{2} + 12.0 s + 8.0}{s^{3} + 3 s^{2} + 5.0 s + 3.0}$

This was a pretty challenging problem. The last time I did a problem like this was 44 years ago when I was taking a graduate level EE class on Network Synthesis. Back then the IBM PC hadn't been invented yet. Over the course of my career I have designed many filters, but I usually just followed the cook book method.   